# Cross-validation error and its variance decomposition

Cross-validation error as any empirical risk estimator depends on the observed data.
The magnitude of statistical fluctuations is characterised by its variance – we can expect that for large enough datasets, the cross-validation error is distributed similarly to a normal distribution.

The paper *No Unbiased Estimator for the Variance of K-Fold Cross-Validation* by *Yoshua Bengio* and *Yves Grandvalet* provides a nice decomposition result for the variance of cross-validation error
\begin{align*}
\theta= \frac{1}{n}\cdot\sigma^2+ \frac{m-1}{n}\cdot \omega + \frac{n-m}{n}\cdot\gamma,
\end{align*}
where
* $\sigma^2$ is the average variance of a true test error 
* $\omega$ is within-block covariance of test errors
* $\gamma$ is between-block covariance of test errors

As it is quite difficult to formally define without specifying an algorithm, we define these in terms of Python code instead of a formal definition.  

In [10]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn

from pandas import Series
from pandas import DataFrame
from typing import Tuple

from tqdm.notebook import tnrange
from sklearn.linear_model import LogisticRegression
from plotnine import *

In [6]:
import sys
sys.path.append('../')
from common import logit
from common import data_sampler
from common import MajorityVoting

## I. Experiment setup

We again consider a relatively simple prediction task $\mathcal{D}_1$ with a relatively small feature set and an impossible prediction task $\mathcal{D}_0$ with the same feature set for comparison. 
We use majority voting and logistic regression as example classifiers as in the previous notebook. 

In [3]:
sampler_0 = lambda n: data_sampler(n, 8, lambda x: logit(x, Series([0, 0])))
sampler_1 = lambda n: data_sampler(n, 8, lambda x: logit(x, Series([10, 10])))
clf_1 = MajorityVoting()
clf_2 = LogisticRegression(solver = 'lbfgs')

## II. Average variance of a true test error

Recall that k-fold cross-validation splits the data into $k$ folds, each of size $m$.
The true variance $\sigma^2$ measures the variance of a test error averaged over all possible training sets of size $(k-1)m$. Note that this term is well defined due to symmetry:

* All splits are equivalent.
* It does not matter which datapoints form the test set we take.

The following algorithm would provide the true value for $\sigma^2$ in the process $r\to\infty$ with probability 1.

In [4]:
def true_error_variance(sampler, clf, features, target, n: int, k:int=10, r:int=1000):
    assert n % k == 0,  'Crossvalidation is unimplemented for cases n != k * m'
   
    m = int(n/k)

    # Record losses of a data point in a test fold over all possible training sets 
    loss = Series(np.nan, index=range(r))
    for i in tnrange(r):
        
        # Train a model
        train = sampler((k-1) * m)
        test  = sampler(m)
        clf.fit(train[features], train[target])
    
        # We record the variance of the first test sample 
        loss[i] = float((clf.predict(test[features]) != test[target])[0])
    
    
    return loss.var()

### True variance component  for $\mathcal{D}_0$ and majority voting

In [11]:
k = 10
n = 100
r = 1000
features = list(sampler_0(1).columns.values[:-1])
var_1 = true_error_variance(sampler_0, clf_1, features, 'y', n, k, r)
var_2 = true_error_variance(sampler_0, clf_1, features, 'y', n, k, r)
print('Sigma^2 for n = {}\n{:+.6}\n{:+.6}'.format(n, var_1, var_2))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Sigma^2 for n = 100
+0.250225
+0.250234


### True variance component  for $\mathcal{D}_1$ and logistic regression

In [12]:
k = 10
n = 100
r = 1000
features = list(sampler_1(1).columns.values[:-1])
var_1 = true_error_variance(sampler_1, clf_2, features, 'y', n, k, r)
var_2 = true_error_variance(sampler_1, clf_2, features, 'y', n, k, r)
print('Sigma^2 for n = {}\n{:+.6}\n{:+.6}'.format(n, var_1, var_2))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Sigma^2 for n = 100
+0.200024
+0.189674


## III. Within-block covariance

Within-block covariance $\omega$ measures the covariance of two different error terms in the same test fold.
Again, the covariance value is computed over all possible cross-validation sets.
Note that this term is well defined due to symmetry:
* All splits are equivalent.
* It does not matter which datapoints form the test set we take.

The following algorithm would provide the true value for $\omega$ in the process $r\to\infty$ with probability 1.

In [13]:
def within_block_covariance(sampler, clf, features, target, n: int, k:int=10, r:int=1000):
    assert n % k == 0,  'Crossvalidation is unimplemented for cases n != k * m'
   
    m = int(n/k)
    
    # There is only one error 
    if m <= 1:
        return 0

    # Record losses of two different data points belonging to the same test fold over all the training sets. 
    loss_1 = Series(np.nan, index=range(r))
    loss_2 = Series(np.nan, index=range(r))
    for i in tnrange(r):
        
        # Train a model
        train = sampler((k-1) * m)
        test = sampler(m)
        clf.fit(train[features], train[target])
    
        # We record the loss of the first and the second test sample in the split
        yp = clf.predict(test[features])
        loss_1[i] = float((yp != test[target])[0])
        loss_2[i] = float((yp != test[target])[1])
                
    # Compute the covariance between loss vectors              
    return loss_1.cov(loss_2)

### Within-block covariance component  for $\mathcal{D}_0$ and majority voting

Within-block covariance component $\omega$ must be zero for all prediction algorithms as true target values are independent, so the errors are independent.

In [14]:
k = 10
n = 100
r = 1000
features = list(sampler_0(1).columns.values[:-1])
omega_1 = within_block_covariance(sampler_0, clf_1, features, 'y', n, k, r)
omega_2 = within_block_covariance(sampler_0, clf_1, features, 'y', n, k, r)
print('Omega for n = {}\n{:+.6}\n{:+.6}'.format(n, omega_1, omega_2))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Omega for n = 100
+0.00446847
+0.0107267


### Within-block covariance component  for $\mathcal{D}_1$ and logistic regression

As logistic regression quickly learns the true model on training set, test set errors become independent again.

In [15]:
k = 10
n = 100
r = 1000
features = list(sampler_0(1).columns.values[:-1])
omega_1 = within_block_covariance(sampler_1, clf_2, features, 'y', n, k, r)
omega_2 = within_block_covariance(sampler_1, clf_2, features, 'y', n, k, r)
print('Omega for n = {}\n{:+.6}\n{:+.6}'.format(n, omega_1, omega_2))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Omega for n = 100
-0.0104424
+0.00511712


## III. Between-block covariance

Between-block covariance measures the covariance of two error terms in different test folds.
Again, the covariance value is computed over all possible cross-validation sets.
Note that this term is well defined due to symmetry:
* All split pairs are equivalent.
* It does not matter which datapoints form the test set we take.

The following algorithm would provide the true value for $\gamma$ in the process $r\to\infty$ with probability 1.

In [16]:
def between_block_covariance(sampler, clf, features, target, n: int, k:int=10, r:int=100):
    assert n % k == 0,  'Crossvalidation is unimplemented for cases n != k * m'
   
    m = int(n/k)

    # Record losses of two data points belonging to different test folds over all the training sets. 
    loss_1 = Series(np.nan, index=range(r))
    loss_2 = Series(np.nan, index=range(r))
    for i in tnrange(r):
        
        # As two splits are overlapping they have joint training set of size (k-2)*m 
        joint_train = sampler((k-2) * m)
        test_1 = sampler(m)
        test_2 = sampler(m)
        
        # We record the loss of the first test sample in the first split
        train = pd.concat([joint_train, test_2])
        clf.fit(train[features], train[target])
        loss_1[i] = float(clf.predict(test_1[features])[0] != test_1[target][0])

        # We record the loss of the first test sample in the second split
        train = pd.concat([joint_train, test_1])
        clf.fit(train[features], train[target])
        loss_2[i] = float(clf.predict(test_2[features])[0] !=test_2[target][0])
    
    # Compute the covariance between loss vectors  
    return loss_1.cov(loss_2)

### Between-block covariance component for $\mathcal{D}_0$ and majority voting

In [17]:
k = 10
n = 100
r = 1000
features = list(sampler_0(1).columns.values[:-1])
gamma_1 = between_block_covariance(sampler_0, clf_1, features, 'y', n, k, r)
gamma_2 = between_block_covariance(sampler_0, clf_1, features, 'y', n, k, r)
print('Gamma for n = {}\n{:+.6}\n{:+.6}'.format(n, gamma_1, gamma_2))

  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Gamma for n = 100
+0.00200901
-0.00456456


### Between-block covariance component for $\mathcal{D}_1$ and logistic regression 


In [18]:
k = 10
n = 100
r = 1000
features = list(sampler_0(1).columns.values[:-1])
gamma_1 = between_block_covariance(sampler_1, clf_2, features, 'y', n, k, r)
gamma_2 = between_block_covariance(sampler_1, clf_2, features, 'y', n, k, r)
print('Gamma for n = {}\n{:+.6}\n{:+.6}'.format(n, gamma_1, gamma_2))


  0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Gamma for n = 100
+0.00726727
-0.000694695


## IV. Empirical test for the theorem

To be completed

## Examples of cross-validation variance decomposition* 

The examples above were not very convincing as $\omega$ and $\gamma$ values were too close to zero. 
Find examples where this is not the case. You can take the paper *No Unbiased Estimator for the Variance of K-Fold Cross-Validation* as a starting point since it contains several examples of such datasets, both artificial and real.
Implement either their artificial samplers or find a UCI repository dataset that is similar to the **Letter** dataset.

In [12]:
%config IPCompleter.greedy=True